In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gdsc-data-extra/gdsc_data_extra.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import time
from matplotlib.cm import viridis
import warnings
warnings.filterwarnings('ignore')

In [3]:
pip install kneed

Note: you may need to restart the kernel to use updated packages.


In [4]:
gdsc_data = pd.read_csv('/kaggle/input/gdsc-data-extra/gdsc_data_extra.csv') 
print(f"Dataset shape: {gdsc_data.shape}")
gdsc_data.head().T

Dataset shape: (198342, 21)


,0,1,2,3,4
COSMIC_ID,683667,687448,687452,687455,687457
CELL_LINE_NAME,PFSK-1,COLO-829,5637,RT4,SW780
TCGA_DESC,MB,SKCM,BLCA,BLCA,BLCA
DRUG_ID,1003,1003,1003,1003,1003
DRUG_NAME,Camptothecin,Camptothecin,Camptothecin,Camptothecin,Camptothecin
LN_IC50,-1.463887,-1.235034,-2.632632,-2.963191,-1.449138
AUC,0.93022,0.867348,0.834067,0.821438,0.90505
Z_SCORE,0.433123,0.557727,-0.203221,-0.3832,0.441154
GDSC Tissue descriptor 1,nervous_system,skin,Unknown,urogenital_system,urogenital_system
GDSC Tissue descriptor 2,medulloblastoma,melanoma,Unknown,Bladder,Bladder


In [5]:
features = ['AUC', 'Z_SCORE', 'Drug_Potency_Index', 'Response_Variability']
X = gdsc_data[features]
y = gdsc_data['LN_IC50']

In [6]:
numerical_features_withLabel = ['LN_IC50', 'AUC', 'Z_SCORE', 'Drug_Potency_Index', 'Response_Variability']
numerical_features = gdsc_data[numerical_features_withLabel]
numerical_features.to_csv('numerical_features.csv', index=False)

In [7]:
numerical_features.head()

,LN_IC50,AUC,Z_SCORE,Drug_Potency_Index,Response_Variability
0,-1.463887,0.930220,0.433123,4.037985,0.390564
1,-1.235034,0.867348,0.557727,3.558215,0.448556
2,-2.632632,0.834067,-0.203221,5.236444,0.262146
3,-2.963191,0.821438,-0.383200,5.619105,0.527685
4,-1.449138,0.905050,0.441154,3.943711,0.395158


In [8]:
print("\nFeatures for clustering:")
print(X.head())


Features for clustering:
        AUC   Z_SCORE  Drug_Potency_Index  Response_Variability
0  0.930220  0.433123            4.037985              0.390564
1  0.867348  0.557727            3.558215              0.448556
2  0.834067 -0.203221            5.236444              0.262146
3  0.821438 -0.383200            5.619105              0.527685
4  0.905050  0.441154            3.943711              0.395158


In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_df = pd.DataFrame(X_scaled, columns=features)

In [10]:
n_features = ['LN_IC50', 'AUC', 'Z_SCORE', 'Drug_Potency_Index', 'Response_Variability']
scaler1 = StandardScaler()
numerical_features_scaled = scaler1.fit_transform(numerical_features)
numerical_features_scaled_df = pd.DataFrame(numerical_features_scaled, columns=n_features)
numerical_features_scaled_df.head()

,LN_IC50,AUC,Z_SCORE,Drug_Potency_Index,Response_Variability
0,-2.527836,0.075527,0.360484,2.326585,-0.709719
1,-2.409852,-0.817682,0.500502,2.092022,-0.647003
2,-3.130381,-1.290498,-0.354576,2.912521,-0.848600
3,-3.300801,-1.469916,-0.556818,3.099607,-0.561426
4,-2.520233,-0.282058,0.369508,2.280494,-0.704752


In [11]:
numerical_features_scaled_df.to_csv('numerical_features_scaled_df.csv', index=False)

In [12]:
X_df.head()

,AUC,Z_SCORE,Drug_Potency_Index,Response_Variability
0,0.075527,0.360484,2.326585,-0.709719
1,-0.817682,0.500502,2.092022,-0.647003
2,-1.290498,-0.354576,2.912521,-0.848600
3,-1.469916,-0.556818,3.099607,-0.561426
4,-0.282058,0.369508,2.280494,-0.704752


In [13]:
norm = plt.Normalize(y.min(), y.max())
colors = viridis(norm(y))

In [14]:
def silhouette_analysis_with_minibatch():
    k_range = range(2, 11)
    silhouette_avg = []
    
    # Sample for silhouette calculation
    sample_size = min(10000, int(0.05 * len(X_scaled)))
    random_indices = np.random.choice(len(X_scaled), sample_size, replace=False)
    X_sample = X_scaled[random_indices]
    
    print(f"Running MiniBatchKMeans + silhouette on {sample_size} samples")
    
    start_time = time.time()
    for k in k_range:
        # Use MiniBatchKMeans, which is much faster than regular KMeans
        minibatch_kmeans = MiniBatchKMeans(n_clusters=k, random_state=42, 
                                           batch_size=1024, n_init='auto')
        # Fit on entire dataset
        minibatch_kmeans.fit(X_scaled)
        # Predict on the sample
        cluster_labels = minibatch_kmeans.predict(X_sample)
        # Calculate silhouette score on the sample
        silhouette_avg.append(silhouette_score(X_sample, cluster_labels))
        print(f"k={k} completed in {time.time() - start_time:.2f} seconds")
        start_time = time.time()
    
    return k_range, silhouette_avg

In [15]:
k_range, silhouette_avg = silhouette_analysis_with_minibatch()

Running MiniBatchKMeans + silhouette on 9917 samples
k=2 completed in 1.58 seconds
k=3 completed in 1.33 seconds
k=4 completed in 1.31 seconds
k=5 completed in 1.27 seconds
k=6 completed in 1.27 seconds
k=7 completed in 1.25 seconds
k=8 completed in 1.28 seconds
k=9 completed in 1.31 seconds
k=10 completed in 1.27 seconds


In [16]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_avg, 'o-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Average Silhouette Score')
plt.title('Silhouette Analysis for Optimal k')
plt.grid(True)
plt.savefig('silhouette_analysis.png')
plt.close()

In [17]:
best_k_indices = np.argsort(silhouette_avg)[-3:]
best_k_values = [k_range[i] for i in best_k_indices]
print(f"Best k values based on silhouette analysis: {best_k_values}")

Best k values based on silhouette analysis: [2, 4, 3]


In [18]:
# Perform K-means clustering for the three best k values
for k in best_k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    gdsc_data['kmeans_cluster'] = kmeans.fit_predict(X_scaled)
    centroids = kmeans.cluster_centers_
    
    # Create a 2D visualization using the first two features
    plt.figure(figsize=(12, 10))
    
    # Plot samples colored by LN_IC50
    scatter = plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y, cmap='viridis', 
                          alpha=0.6, edgecolor='k', s=50)
    
    # Plot centroids
    plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200, 
                edgecolor='k', label='Centroids')
    
    plt.colorbar(scatter, label='LN_IC50')
    plt.title(f'K-means Clustering (k={k})')
    plt.xlabel(features[0])
    plt.ylabel(features[1])
    plt.legend()
    plt.savefig(f'kmeans_k{k}.png')
    plt.close()
    
    # Calculate and display distribution of target variable per cluster
    cluster_stats = gdsc_data.groupby('kmeans_cluster')['LN_IC50'].agg(['mean', 'std', 'count'])
    print(f"\nCluster statistics for k={k}:")
    print(cluster_stats)


Cluster statistics for k=2:
                    mean       std   count
kmeans_cluster                            
0               2.608734  1.645872  133777
1               5.160238  1.246684   64565

Cluster statistics for k=4:
                    mean       std  count
kmeans_cluster                           
0               2.827848  1.170628  30897
1               1.030084  1.363876  40707
2               3.903628  1.313326  88926
3               5.440643  1.220716  37812

Cluster statistics for k=3:
                    mean       std   count
kmeans_cluster                            
0               3.707740  1.281583  107600
1               1.190558  1.352907   49101
2               5.397299  1.239226   41641


In [19]:
sample_size = min(5000, len(X_scaled))
indices = np.random.choice(range(len(X_scaled)), sample_size, replace=False)
X_sample = X_scaled[indices]
y_sample = y.iloc[indices]

In [20]:
# Calculate cosine similarity on the sample
print("Computing distance matrix...")
cosine_distances = pdist(X_sample, metric='cosine')

Computing distance matrix...


In [21]:
# Perform hierarchical clustering
print("Computing linkage...")
Z = linkage(cosine_distances, method='average')

# Plot dendrogram
plt.figure(figsize=(16, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Sample index')
plt.ylabel('Distance (Cosine)')

# Make dendrogram more readable by truncating
dendrogram(Z, truncate_mode='lastp', p=30, leaf_font_size=10, 
          color_threshold=0.7 * max(Z[:, 2]))

plt.savefig('hierarchical_clustering_dendrogram.png')
plt.close()

Computing linkage...


In [22]:
# Cut the dendrogram to get clusters (using same number as best k from k-means)
for k in best_k_values:
    hierarchical_clusters = fcluster(Z, k, criterion='maxclust')
    
    # Plot 2D visualization of hierarchical clusters
    plt.figure(figsize=(12, 10))
    plt.scatter(X_sample[:, 0], X_sample[:, 1], c=hierarchical_clusters, 
                cmap='viridis', alpha=0.6, edgecolor='k', s=50)
    plt.title(f'Hierarchical Clustering (k={k})')
    plt.xlabel(features[0])
    plt.ylabel(features[1])
    plt.colorbar(label='Cluster')
    plt.savefig(f'hierarchical_k{k}.png')
    plt.close()

In [23]:
# DBSCAN Clustering
from sklearn.neighbors import NearestNeighbors

sample_size_knn = min(10000, len(X_scaled))
random_indices_knn = np.random.choice(len(X_scaled), sample_size_knn, replace=False)
X_sample_knn = X_scaled[random_indices_knn]
print("Performing DBSCAN Clustering...")

sample_size_knn = min(10000, len(X_scaled))
random_indices_knn = np.random.choice(len(X_scaled), sample_size_knn, replace=False)
X_sample_knn = X_scaled[random_indices_knn]

Performing DBSCAN Clustering...


In [24]:
# Calculate distances to 5th nearest neighbor
neighbors = NearestNeighbors(n_neighbors=5)
neighbors.fit(X_sample_knn)
distances, _ = neighbors.kneighbors(X_sample_knn)
distances = np.sort(distances[:, 4]) 

In [25]:
# Plotting k-distance graph
plt.figure(figsize=(10, 6))
plt.plot(distances)
plt.ylabel('Distance to 5th nearest neighbor')
plt.xlabel('Points (sorted)')
plt.title('K-distance Graph for DBSCAN Epsilon Parameter Selection')
plt.grid(True)
plt.savefig('dbscan_kdistance.png')
plt.close()

In [26]:
# Find the "elbow" point
from kneed import KneeLocator
kneedle = KneeLocator(range(len(distances)), distances, S=1.0, 
                     curve='convex', direction='increasing')
eps = distances[kneedle.knee]
print(f"Estimated epsilon: {eps}")

# Run DBSCAN with optimized parameters
dbscan = DBSCAN(eps=eps, min_samples=5)
dbscan_sample_size = min(50000, len(X_scaled))
random_indices_dbscan = np.random.choice(len(X_scaled), dbscan_sample_size, replace=False)
X_sample_dbscan = X_scaled[random_indices_dbscan]
y_sample_dbscan = y.iloc[random_indices_dbscan]

dbscan_labels = dbscan.fit_predict(X_sample_dbscan)

Estimated epsilon: 0.5656658525046282


In [27]:
# Plot DBSCAN results
plt.figure(figsize=(12, 10))
unique_labels = np.unique(dbscan_labels)
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))

for label, col in zip(unique_labels, colors):
    if label == -1:  # Noise points
        col = 'black'
    mask = dbscan_labels == label
    plt.scatter(X_sample_dbscan[mask, 0], X_sample_dbscan[mask, 1], 
               c=[col], label=f'Cluster {label}' if label != -1 else 'Noise',
               s=50, alpha=0.6, edgecolor='k')

plt.title('DBSCAN Clustering')
plt.xlabel(features[0])
plt.ylabel(features[1])
plt.legend()
plt.savefig('dbscan_clustering.png')
plt.close()

In [28]:
# Counting number of points in each cluster and noise
cluster_counts = np.bincount(dbscan_labels[dbscan_labels >= 0])
noise_count = np.sum(dbscan_labels == -1)
print(f"Number of clusters found by DBSCAN: {len(cluster_counts)}")
print(f"Number of noise points: {noise_count} ({noise_count/len(dbscan_labels)*100:.2f}%)")

Number of clusters found by DBSCAN: 7
Number of noise points: 25 (0.05%)


In [29]:
# Calculate statistics for each DBSCAN cluster
dbscan_df = pd.DataFrame({
    'dbscan_cluster': dbscan_labels,
    'LN_IC50': y_sample_dbscan.values
})

for label in unique_labels:
    cluster_data = dbscan_df[dbscan_df['dbscan_cluster'] == label]
    label_name = 'Noise' if label == -1 else f'Cluster {label}'
    print(f"\n{label_name} statistics:")
    print(f"Count: {len(cluster_data)}")
    if len(cluster_data) > 0:
        print(f"Mean LN_IC50: {cluster_data['LN_IC50'].mean():.4f}")
        print(f"Std LN_IC50: {cluster_data['LN_IC50'].std():.4f}")


Noise statistics:
Count: 25
Mean LN_IC50: 1.7531
Std LN_IC50: 3.4260

Cluster 0 statistics:
Count: 49735
Mean LN_IC50: 3.4239
Std LN_IC50: 1.9020

Cluster 1 statistics:
Count: 37
Mean LN_IC50: -2.5078
Std LN_IC50: 0.4973

Cluster 2 statistics:
Count: 164
Mean LN_IC50: 8.8495
Std LN_IC50: 0.4491

Cluster 3 statistics:
Count: 8
Mean LN_IC50: 0.1707
Std LN_IC50: 0.2887

Cluster 4 statistics:
Count: 12
Mean LN_IC50: -2.6728
Std LN_IC50: 0.3557

Cluster 5 statistics:
Count: 14
Mean LN_IC50: 7.6511
Std LN_IC50: 0.3752

Cluster 6 statistics:
Count: 5
Mean LN_IC50: -3.1678
Std LN_IC50: 0.0685


In [30]:
# Compare clustering results
print("\nComparison of Clustering Methods:")
print("1. K-means identified spherical clusters based on distance to centroids")
print("2. Hierarchical clustering grouped samples based on cosine similarity")
print(f"3. DBSCAN found {len(cluster_counts)} clusters and {noise_count} noise points based on density")


Comparison of Clustering Methods:
1. K-means identified spherical clusters based on distance to centroids
2. Hierarchical clustering grouped samples based on cosine similarity
3. DBSCAN found 7 clusters and 25 noise points based on density


In [31]:
# Create a mapping from the DBSCAN sample back to the original indices
sample_indices_map = {i: idx for i, idx in enumerate(random_indices_dbscan)}

# Initialize an array for all data points (default to -1 for noise)
all_dbscan_labels = np.full(len(gdsc_data), -1)

# Fill in the labels for the sampled points
for i, label in enumerate(dbscan_labels):
    original_idx = random_indices_dbscan[i]
    all_dbscan_labels[original_idx] = label

# Add DBSCAN cluster labels to the main dataframe
gdsc_data['dbscan_cluster'] = all_dbscan_labels

# Now calculate variance explained by each clustering method
kmeans_variance = gdsc_data.groupby('kmeans_cluster')['LN_IC50'].var().sum() / gdsc_data['LN_IC50'].var()
dbscan_variance = gdsc_data[gdsc_data['dbscan_cluster'] >= 0].groupby('dbscan_cluster')['LN_IC50'].var().sum() / gdsc_data['LN_IC50'].var()

print("\nVariance Explained in Drug Sensitivity (LN_IC50):")
print(f"K-means: {(1 - kmeans_variance) * 100:.2f}%")
print(f"DBSCAN: {(1 - dbscan_variance) * 100:.2f}%")

# For hierarchical clustering, create a similar mapping
# First initialize an array for all data points
all_hierarchical_labels = np.full(len(gdsc_data), -1)

# Fill in the labels for the sampled points
for i, label in enumerate(hierarchical_clusters):
    original_idx = indices[i]
    all_hierarchical_labels[original_idx] = label

# Add hierarchical cluster labels to the main dataframe
gdsc_data['hierarchical_cluster'] = all_hierarchical_labels

# Calculate variance explained by hierarchical clustering
hierarchical_variance = gdsc_data[gdsc_data['hierarchical_cluster'] >= 0].groupby('hierarchical_cluster')['LN_IC50'].var().sum() / gdsc_data['LN_IC50'].var()
print(f"Hierarchical: {(1 - hierarchical_variance) * 100:.2f}%")


Variance Explained in Drug Sensitivity (LN_IC50):
K-means: -33.12%
DBSCAN: -17.53%
Hierarchical: -17.65%
